In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec, FastText
import spacy

2023-11-12 23:34:35.347405: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 23:34:35.347433: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 23:34:35.347462: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 23:34:35.353669: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# MEDICAL NER EXTRACTION USING MED7

In [3]:
med7 = spacy.load("en_core_med7_lg")
new_notes = pd.read_pickle("../data/clinical_notes/preprocessed_notes.p")
new_notes['ner'] = None

In [5]:
count = 0
preprocessed_index = {}
for i in new_notes.itertuples():

    if count % 1000 == 0:
        print(count)

    count += 1
    ind = i.Index
    text = i.preprocessed_text

    all_pred = []
    for each_sent in text:
        try:
            doc = med7(each_sent)
            result = ([(ent.text, ent.label_) for ent in doc.ents])
            if len(result) == 0: continue
            all_pred.append(result)
        except:
            print("error..")
            continue
    new_notes.at[ind, 'ner'] = all_pred

pd.to_pickle(new_notes, "../data/clinical_notes/ner_df.p")

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000


In [5]:
def mean(a):
    return sum(a) / len(a)

In [6]:
##Patient ID -> NER words dictionary
null_index_list = []
for i in new_notes.itertuples():

    if len(i.ner) == 0:
        null_index_list.append(i.Index)
new_notes.drop(null_index_list, inplace=True)

med7_ner_data = {}

for ii in new_notes.itertuples():

    p_id = ii.SUBJECT_ID
    ind = ii.Index

    try:
        new_ner = new_notes.loc[ind].ner
    except:
        new_ner = []

    unique = set()
    new_temp = []

    for j in new_ner:
        for k in j:
            unique.add(k[0])
            new_temp.append(k)

    if p_id in med7_ner_data:
        for i in new_temp:
            med7_ner_data[p_id].append(i)
    else:
        med7_ner_data[p_id] = new_temp

pd.to_pickle(med7_ner_data, "../data/clinical_notes/new_ner_word_dict.pkl")

# WORD2VEC EMBEDDING REPRESENTATION

In [8]:
w2vec = Word2Vec.load("../embeddings/word2vec.model")

In [9]:
data_types = [med7_ner_data]
data_names = ["new_ner"]
new_word2vec = {}

for data, names in zip(data_types, data_names):
    print("w2vec starting..")
    for k,v in data.items():

        patient_temp = []
        for i in v:
            if w2vec.wv.has_index_for(i[0]):
                patient_temp.append(w2vec.wv[i[0]])
            else:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        if w2vec.wv.has_index_for(each_word):
                            temp = w2vec.wv[each_word]
                            avg.append(temp)
                            num += 1
                    if num == 0: continue
                    avg = np.asarray(avg)
                    t = np.asarray(list(map(mean, zip(*avg))))
                    patient_temp.append(t)
        if len(patient_temp) == 0: continue
        new_word2vec[k] = patient_temp

    print("word2vec length is {}", len(new_word2vec))
    pd.to_pickle(new_word2vec, "../data/clinical_notes/representations/" + names + "_word2vec_dict.pkl")

w2vec starting..
word2vec length is {} 22203


# FASTTEXT EMBEDDING REPRESENTATION

In [13]:
fasttext = FastText.load_fasttext_format("../embeddings/fasttext.bin")

/tmp/ipykernel_274270/3588191094.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  fasttext = FastText.load_fasttext_format("../embeddings/fasttext.bin")


In [14]:
data_types = [med7_ner_data]
data_names = ["new_ner"]
new_fasttextvec = {}

for data, names in zip(data_types, data_names):
    print("fasttext starting..")        
    for k,v in data.items():
        patient_temp = []
        for i in v:
            if fasttext.wv.has_index_for(i[0]):
                patient_temp.append(fasttext.wv[i[0]])
        if len(patient_temp) == 0: continue
        new_fasttextvec[k] = patient_temp
    print("fasttext length is {}", len(new_fasttextvec))
    pd.to_pickle(new_fasttextvec, "../data/clinical_notes/representations/" + names + "_fasttext_dict.pkl")
 

fasttext starting..
fasttext length is {} 22327


# CONCAT EMBEDDING REPRESENTATION

In [20]:
data_types = [med7_ner_data]
data_names = ["new_ner"]
new_concatvec = {}

for data, names in zip(data_types, data_names):
    print("concat starting..")        
    for k,v in data.items():
        if k in new_word2vec and k in new_fasttextvec:
            patient_temp = []
            for i in v:
                w2vec_temp = []
                if w2vec.wv.has_index_for(i[0]):
                    w2vec_temp = w2vec.wv[i[0]]
                else:
                    avg = []
                    num = 0
                    temp = []

                    if len(i[0].split(" ")) > 1:
                        for each_word in i[0].split(" "):
                            if w2vec.wv.has_index_for(each_word):
                                temp = w2vec.wv[each_word]
                                avg.append(temp)
                                num += 1
                        if num == 0: 
                            w2vec_temp = [0] * 100
                        else:
                            avg = np.asarray(avg)
                            w2vec_temp = np.asarray(list(map(mean, zip(*avg))))
                    else:
                        w2vec_temp = [0] * 100
                
                if fasttext.wv.has_index_for(i[0]):
                    fasttemp = fasttext.wv[i[0]]
                else:
                    fasttemp = [0] * 100

                appended = np.append(fasttemp, w2vec_temp, 0)
                patient_temp.append(appended)
            if len(patient_temp) == 0: continue
            new_concatvec[k] = patient_temp

    print("concat length is {}", len(new_concatvec))
    pd.to_pickle(new_fasttextvec, "../data/clinical_notes/representations/" + names + "_concat_dict.pkl")

concat starting..
concat length is {} 22095


In [21]:
print(len(new_word2vec), len(new_fasttextvec), len(new_concatvec))

22203 22327 22095


In [18]:
intersect = set(new_word2vec.keys()) & set(new_fasttextvec.keys()) & set(new_concatvec.keys())
new_word2vec_dict = {x: new_word2vec[x] for x in new_word2vec if x in intersect}
new_fasttextvec_dict = {x: new_fasttextvec[x] for x in new_fasttextvec if x in intersect}
new_concatvec_dict = {x: new_concatvec[x] for x in new_concatvec if x in intersect}
print (len(new_word2vec_dict), len(new_fasttextvec_dict), len(new_concatvec_dict))


pd.to_pickle(new_word2vec_dict, "../data/clinical_notes/representations/"+"new_ner"+"_word2vec_limited_dict.pkl")
pd.to_pickle(new_fasttext_dict, "../data/clinical_notes/representations/"+"new_ner"+"_fasttext_limited_dict.pkl")
pd.to_pickle(new_combined_dict, "../data/clinical_notes/representations/"+"new_ner"+"_combined_limited_dict.pkl")

NameError: name 'new_fasttext_dict' is not defined